In [1]:
from langchain_community.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import re

import os
from dotenv import load_dotenv


Importamos el API Key de las variables de entorno

In [2]:
load_dotenv()

os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')

In [3]:
# pip install langchain_community

In [4]:
# pip install pypdf

In [5]:
# pip install sentence-transformers

In [6]:
# !pip install chromadb

In [7]:
# pip install python-dotenv

In [8]:
# pip install google-generativeai


# Leer el documentos

In [9]:
def cargar_documentos(ruta_archivo):
    """Carga documentos en PDF, TXT o DOCX y los convierte en texto."""
    if ruta_archivo.endswith(".pdf"):
        loader = PyPDFLoader(ruta_archivo)
    elif ruta_archivo.endswith(".txt"):
        loader = TextLoader(ruta_archivo)
    elif ruta_archivo.endswith(".docx"):
        loader = Docx2txtLoader(ruta_archivo)
    else:
        raise ValueError("Formato no soportado. Usa PDF, TXT o DOCX.")
    
    documentos = loader.load()
    
    return documentos

In [10]:
# Función de limpieza
def clean_text(text):
    # Eliminar fechas en formato dd/mm/yyyy
    text = re.sub(r'\d{2}/\d{2}/\d{4}', '', text)
    
    # Eliminar metadatos innecesarios
    text = re.sub(r'USUARIO|PScript5\.dll.*?\n|Acrobat Distiller.*?\n', '', text)
    
    # Reemplazar caracteres especiales y símbolos unicode, excluyendo caracteres españoles
    text = re.sub(r'[\uf06e\uf0a7]|[^\x00-\x7F\xC0-\xFF]+', '-', text)
    
    # Eliminar múltiples espacios en blanco
    text = re.sub(r'\s+', ' ', text)
    
    # Eliminar saltos de línea innecesarios, pero mantener párrafos
    text = re.sub(r'\n{3,}', '\n\n', text)
    
    # Eliminar espacios al inicio y final de cada línea
    text = '\n'.join(line.strip() for line in text.split('\n'))
    
    # Eliminar espacios en blanco al inicio y final del texto
    text = text.strip()
    
    # Eliminar caracteres especiales y símbolos repetidos
    text = re.sub(r'[-]{2,}', '-', text)
    text = re.sub(r'[.]{2,}', '.', text)
    
    return text

# page = cargar_documentos("data/1-01-Curso_PLN.pdf")

# # Limpiar cada página
# cleaned_pages = [clean_text(pag.page_content) for pag in page]

# # Unir todas las páginas en un solo texto limpio
# final_text = "\n\n".join(cleaned_pages)

# # Mostrar el resultado limpio
# print(final_text)


# Split document

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# Paso 1: Convertir el texto limpio en un objeto Document
# Asegúrate de que final_text es un string que contiene el contenido limpio
# document = Document(page_content=final_text)  # Debe ser un objeto Document

# Paso 2: Función para dividir el texto en fragmentos
def split_text(document):
    """Divide el texto en fragmentos más pequeños para procesamiento."""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,      # Número de caracteres por fragmento
        chunk_overlap=50,    # Traslape entre fragmentos
        length_function=len, # Función de longitud
        separators=["\n\n", "\n", " "]  # Separadores
    )

    # Aplicar el splitter al documento (documento debe ser una lista)
    textos_fragmentados = text_splitter.split_documents([document])  # Pasamos una lista de documentos

    return textos_fragmentados

# Paso 3: Ejecutar la función y obtener los fragmentos
# chunks = split_text(document)

# Mostrar un fragmento de ejemplo
# for i, chunk in enumerate(chunks[:]):  # Mostramos solo los 3 primeros
#     print(f"\nFragmento {i+1}:\n{chunk.page_content}\n{'-'*50}")

# Crear embeddings

Se hará uso de un modelo de Hugging Face all-MiniLM-L6-v2

In [12]:
# Usar modelo de Hugging Face
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Almacenar  embeddings en ChromaDB

In [13]:
import uuid

def create_vectorstore(chunks, embedding_function, vectorstore_path):

    # Lista de valores unicos para documentos
    ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in chunks]
    
    unique_ids = set()
    unique_chunks = []
    
    unique_chunks = [] 
    for chunk, id in zip(chunks, ids):     
        if id not in unique_ids:       
            unique_ids.add(id)
            unique_chunks.append(chunk) 

    #Crea una database de chroma
    vectorstore = Chroma.from_documents(documents=unique_chunks, 
                                        ids=list(unique_ids),
                                        embedding=embedding_function, 
                                        persist_directory = vectorstore_path)

    vectorstore.persist()
    
    return vectorstore

In [14]:
# vectorstore = create_vectorstore(chunks=chunks, 
#                                  embedding_function=embeddings, 
#                                  vectorstore_path="./vectorstore")

# Consulta de datos relevantes

In [15]:
#Cargamos el vectorstore
# database = Chroma(persist_directory="vectorstore",embedding_function=embeddings)

In [16]:
from langchain_core.prompts import ChatPromptTemplate

def prompt_template(vectorstore, title_input):
    #Hacemos el prompt configurado con el retrieve
    PROMPT_TEMPLATE = """
    Eres un asistente para la generación de materiales educativos basados en un programa de curso.
    Utiliza la siguiente información recuperada para crear materiales de aprendizaje estructurados.

    ---

    **Título del curso:** ¿Cuál es el título del curso? {title}

    **Temas principales:** ¿Cuáles son los temas principales que se cubren en el curso? {topics}

    **Objetivos de aprendizaje:** ¿Cuáles son los objetivos de aprendizaje de este curso? {objectives}

    **Lecturas y recursos recomendados:** ¿Cuáles son las lecturas o recursos recomendados para este curso? {resources}

    **Preguntas para discusión:** ¿Cuáles son algunas preguntas para discusión en este curso? {discussion_questions}

    **Ejercicios y problemas de práctica:** ¿Qué ejercicios o problemas de práctica se incluyen en el programa del curso? {practice_problems}

    ---

    Basándote en la información anterior, genera los siguientes materiales educativos:
    1. Notas detalladas de clase.
    2. Problemas de práctica con soluciones.
    3. Preguntas para discusión.
    4. Objetivos de aprendizaje específicos para cada tema.
    5. Lecturas y recursos sugeridos.

    Estructura la respuesta de manera clara y detallada. Si no encuentras algo en el documento puedes utilizar información de internet
    """
    
    # Configure the retriever
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={
            "k": 3
        }
    )

    topics = retriever.invoke("¿Cuáles son los temas principales que se cubren en el curso?")
    objectives = retriever.invoke("¿Cuáles son los objetivos de aprendizaje de este curso?")
    resources = retriever.invoke("¿Cuáles son las lecturas o recursos recomendados para este curso?")
    discussion_questions = retriever.invoke("¿Cuáles son algunas preguntas para discusión en este curso?")
    practice_problems = retriever.invoke("¿Qué ejercicios o problemas de práctica se incluyen en el programa del curso?")

    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    #Insertamos los valores en el prompt
    final_prompt = PROMPT_TEMPLATE.format(
        title=title_input,
        topics=topics,
        objectives=objectives,
        resources=resources,
        discussion_questions=discussion_questions,
        practice_problems=practice_problems
    )

    return final_prompt

# Definimos el LLM

Implementamos el modelo de Gemini

In [17]:
import google.generativeai as genai

def generate_response(final_prompt):
    # Inicializa el modelo Gemini
    model = genai.GenerativeModel('gemini-2.0-flash-001')

    # Genera una respuesta
    response = model.generate_content(final_prompt)

    return response.text


c:\Users\Usuario\Documents\Universidad\Redes Neuronales\Trabajo-4-RN\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Pipeline

In [18]:
def main():
    # Sección de input
    try:
        file_path = input("Introduce la ruta del archivo a procesar (PDF, TXT or DOCX): ").strip()
        if not os.path.exists(file_path):
            raise FileNotFoundError("El archivo no existe.")
    
        # Nombre asignatura
        title = input("Introduce el nombre de la asignatura: ").strip()
        if not title:
            raise ValueError("El nombre de la asignatura no puede estar vacío.")
        
        # Cargar documentos
        documents = cargar_documentos(file_path)

        # Limpiar cada página
        cleaned_pages = [clean_text(pag.page_content) for pag in documents]

        # Unir todas las páginas en un solo texto limpio
        final_text = "\n\n".join(cleaned_pages)

        # Paso 1: Convertir el texto limpio en un objeto Document
        document = Document(page_content=final_text)  # Debe ser un objeto Document

        # Paso 2: Función para dividir el texto en fragmentos
        chunks = split_text(document)

        # Paso 3: embeddings
        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

        # Paso 4: Crear vectorstore
        vectorstore = create_vectorstore(chunks=chunks, 
                                        embedding_function=embeddings, 
                                        vectorstore_path="./vectorstore")
        
        # Paso 5: Crear el prompt
        final_prompt = prompt_template(vectorstore, title)

        # Paso 6: Generar respuesta
        response = generate_response(final_prompt)

        print(response)

    except FileNotFoundError as e:
        print(f"File error: {str(e)}")
    except ValueError as e:
        print(f"Input error: {str(e)}")
    except Exception as e:
        print(f"Error inesperado: {e}")


In [19]:
if __name__ == "__main__":
    main()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_21436\2038879979.py:29: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
Delete of nonexisting embedding ID: 22d2e6b4-a6a3-5aff-869b-c18c2e049867
Delete of nonexisting embedding ID: ec1a055e-5a81-544a-af2d-a95b3eec58a4
Delete of nonexisting embedding ID: 8816a309-1018-5e36-b431-6afa83a35031
Delete of nonexisting embedding ID: 48610763-8dff-5068-93a2-d39aacf89cf1
Delete of nonexisting embedding ID: e0d60b93-f560-5f98-a66b-8a1b2ea2efec
Delete of nonexisting embedding ID: 57ff4786-4701-59f8-ac4a-b20c107df80b
Delete of nonexisti

¡Absolutamente! Aquí tienes un esquema de materiales educativos basados en la información proporcionada, complementada con información estándar en el campo del Análisis y Diseño de Algoritmos:

**Título del Curso:** Análisis y Diseño de Algoritmos

**I. Notas Detalladas de Clase (Esquema General)**

*   **Introducción al Curso**
    *   Motivación: Por qué es importante el análisis y diseño de algoritmos.
    *   Objetivos del curso: Aprender a analizar y diseñar algoritmos eficientes.
    *   Metodología: Clases magistrales, talleres, exposiciones, prácticas.
    *   Consideraciones importantes: Confianza profesor-estudiante, prohibición de copiar código, uso de ayudas disponibles.
*   **Análisis de Algoritmos**
    *   **Fundamentos:**
        *   ¿Qué es un algoritmo? Definición formal.
        *   Criterios de corrección y eficiencia.
        *   Modelos de computación (ej., RAM).
    *   **Notación Asintótica:**
        *   Notación Big-O, Omega, Theta. Definiciones formales y eje